In [1]:
%matplotlib inline
import pandas as pd
import pathlib
import geopandas as gpd
import numpy as np
from matplotlib import pyplot as plt
import mio

In [2]:
def tri_array(p0, p1, p2, area, px, py):
    p0x, p0y = p0
    p1x, p1y = p1
    p2x, p2y = p2
    
    # do the calc, using stolen formula
    s = 1/(2*area)*(p0y*p2x - p0x*p2y + (p2y - p0y)*px + (p0x - p2x)*py)
    t = 1/(2*area)*(p0x*p1y - p0y*p1x + (p0y - p1y)*px + (p1x - p0x)*py)
    u = 1 - s -t
    r = (s>0) & (t>0) & (u>0)
    
    return r.astype(int)

In [3]:
def create_mesh(df):
    # calc boundaries of building
    xmin = int(min(df.x0.min(), df.x1.min(), df.x2.min()) - 1)
    xmax = int(max(df.x0.max(), df.x1.max(), df.x2.max()) + 1)
    ymin = int(min(df.y0.min(), df.y1.min(), df.y2.min()) - 1)
    ymax = int(max(df.y0.max(), df.y1.max(), df.y2.max()) + 1)

    x = np.arange(xmin, xmax + 1, 1)
    y = np.arange(ymin, ymax + 1, 1)
    px, py = np.meshgrid(x, y)
    return px, py

In [4]:
def signed_area(p0, p1, p2):
        # calc signed area of triangle
        p0x, p0y = p0
        p1x, p1y = p1
        p2x, p2y = p2

        return 0.5 *(-p1y*p2x + p0y*(-p1x + p2x) + p0x*(p1y - p2y) + p1x*p2y)

In [5]:
def make_building_raster(source):
    df = pd.read_csv(source)
    px, py = create_mesh(df)

    lr = []
    for ind, row in df.iterrows():
        p0 = row.x0, row.y0
        p1 = row.x1, row.y1
        p2 = row.x2, row.y2

        area = signed_area(p0, p1, p2)
        if area != 0:
            tri_raster = tri_array(p0, p1, p2, area, px, py)
            lr.append(tri_raster)

    res = lr[0]
    for r in lr[1:]:
        res = np.maximum(res, r)
    lon = df.lon.iloc[0]
    lat = df.lat.iloc[0]
    return res, lon, lat

In [6]:
tris = list(pathlib.Path('tri').glob('*.tri'))
len(tris)

13186

In [ ]:
for i, tri in enumerate(tris[0:]):
    mio.show_perc(i, len(tris), 100)
    r, lon, lat = make_building_raster(tri)
    
    x_swiss, y_swiss = mio.wgs_swiss(lon, lat)
    x_swiss, y_swiss = round(x_swiss), round(y_swiss)
    
    dest = 'tif/' + tri.stem + '.tif'
    
    df = pd.DataFrame(r)
    
    ys , xs = df.shape
    
    # TODO: exactly find 0, 0 point
    
    df.columns = df.columns + x_swiss - int(xs / 2)
    df.index = df.index + y_swiss - int(ys / 2)
    df = df.sort_index(ascending=False)
    mio.write_raster((df * 200).astype('uint8'), dest)

0.0% 0.76% 1.52% 2.28% 3.03% 3.79% 4.55% 5.31% 6.07% 6.83% 7.58% 8.34% 9.1% 9.86% 10.62% 11.38% 12.13% 12.89% 13.65% 14.41% 15.17% 15.93% 16.68% 17.44% 18.2% 18.96% 19.72% 20.48% 21.23% 21.99% 22.75% 23.51% 24.27% 25.03% 25.78% 26.54% 27.3% 28.06% 28.82% 29.58% 30.34% 31.09% 31.85% 32.61% 33.37% 34.13% 